In [18]:
import numpy as np
import os
import plyvel as ply
from PIL import Image,ImageOps
import random
import sys
import StringIO


In [19]:
def load_db(db_path, img_size=(224,224)):
    db = ply.DB(db_path, create_if_missing=True)
    imglist = []
    for key, value in db:
        imgitem = value
        img = Image.frombytes('RGB',img_size,imgitem)
        imglist.append(img)

    return imglist

In [20]:
rootpath = "/Users/rwa56/Downloads/tid2013/distorted_images/"
train_file = "/Users/rwa56/project/tidtrain.txt"
db_path = "/Users/rwa56/project/db/"


In [21]:
import time

t0 = time.time()
imgread = load_db(db_path)
t1 = time.time()


In [22]:
t1-t0

0.6274731159210205

In [1]:
from __future__ import absolute_import
import numpy as np
import sys
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cPickle as pickle
import re
from scipy import linalg
import scipy.ndimage as ndi
from six.moves import range
from keras import backend as K
from PIL import Image,ImageOps
import threading
import warnings
import theano

def load_data():
    dirname='/Users/rwa56/Downloads/tid2013/'
    distortion='distorted_images/'
    reference='reference_images/'
    score='mos.txt'

    size= 224, 224


    DistortLabel = np.zeros((3000,3), dtype='float32')
    RefLabel = np.zeros((25,1), dtype='float32')
    label = [0, 0, 0]

    DistortImg = np.zeros((3000, 224,224,3), dtype='uint8')
    RefImg = np.zeros((25,224,224,3), dtype='uint8')
    i = 0

    for root, dirs, filenames in os.walk(dirname+distortion):
        for imgfile in filenames:

            img = Image.open(root+imgfile, 'r')
            #print img.getbands()
            #x = img_to_array(img)
            #img.thumbnail(size, Image.ANTIALIAS)
            img.thumbnail((224,224), Image.ANTIALIAS)

            img = ImageOps.fit(img,(224,224), Image.ANTIALIAS)
            if i == 0:
                img.show()
            #resizeimage.resize_cover(img, [256, 256])
            #ImageOps.fit(img, size, Image.ANTIALIAS)
            #img = tf.image.resize_images(img, size)
           # testimg = np.asarray(img, dtype='float32')
           # print testimg.shape

            DistortImg[i,:,:,:] = np.asarray(img, dtype='float32')
            #x = x.reshape((1,) + x.shape)
            #print 'The imgfile is' +imgfile
        #    x = 0.2126 * x0[:,:,0] + 0.7152 * x0[:,:,1] + 0.0722 * x0[:,:,2]
            #print  x.shape

            if(len(imgfile) < 12):
                print 'The file: ' + imgfile
            DistortLabel[i,0] = int(imgfile[1:3])
            DistortLabel[i,1] = int(imgfile[4:6])
            DistortLabel[i,2] = int(imgfile[7])

            i = i + 1
            #print label


    j = 0

    for root, dirs, filenames in os.walk(dirname+reference):
        for imgfile in filenames:
            #print filename
            img = Image.open(root+imgfile, 'r')
            #print img.getbands()
            #x = img_to_array(img)
            #img.thumbnail(size, Image.ANTIALIAS)
            img.thumbnail((224,224), Image.ANTIALIAS)

            img = ImageOps.fit(img,(224,224), Image.ANTIALIAS)

            RefImg[j,:,:,:] = np.asarray(img, dtype='float32')
            #x = x.reshape((1,) + x.shape)
            #print 'The imgfile is' +imgfile
    #        x = 0.2126 * x0[:,:,0] + 0.7152 * x0[:,:,1] + 0.0722 * x0[:,:,2]

            RefLabel[j] = int(imgfile[1:3])
            #RefLabel.append(label)
            #print label
            j = j + 1


    ScoreLabel = open(dirname+score).read().splitlines()
    ScoreLabel = map(float, ScoreLabel)
    #print ScoreLabel

    #print DistortImg[1]

    return DistortImg, DistortLabel, RefImg, RefLabel, ScoreLabel

Using TensorFlow backend.


In [3]:
import time

t2 = time.time()
DistortImg, DistortLabel, RefImg, RefLabel, ScoreLabel = load_data()
t3 = time.time()

In [4]:
t3-t2

14.511088848114014

In [11]:
import lmdb
def load_db(db_path,imgsize=(224,224)):
    env = lmdb.open(db_path, readonly=True)
    imglist=[]
    with env.begin() as tdb:
        cursor = tdb.cursor()
        for key, value in cursor:
            imgitem = value
            img = Image.frombytes('RGB',imgsize,imgitem)
            imglist.append(img)

In [12]:
lmdb_path = "/Users/rwa56/project/ldb/"

In [13]:
import time

t2 = time.time()
load_db(lmdb_path)
t3 = time.time()

In [14]:
t3-t2

1.9998841285705566